In [ ]:
#in anaconda prompt:
#!conda activate detectron_env

In [ ]:
Assumption: 
# you have the large orthophoto and digitized labels over orthophoto in labelme tool. each solar has different ids (no need to do anything they will have by default)
# saved the file. this will give the json file. You need to convert this json file to coco format json. 

In [ ]:
#convert labelme json to coco json. Use anaconda prompt after activating detectron_env.This will generate the 

In [ ]:
# goto path where you want to save coco json: C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\jsons\coco_json
#labelme2coco "C:\Users\ROG\Documents\Termatics\segmentation\detectron_maskrcnn\jsons\labelme_json"
#Note: make sure you only have one json in the folder json. Otherwise it will also try to convert other files even if they are not json. 

In [ ]:
# Now, you have the coco json and the orthophoto (one big image and coco json (labels) for that)

In [ ]:
#split big image into smaller tiles and generate another json for those smaller tiles in coco format

In [1]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

# --- CONFIG ---
tile_size = 512
overlap = 0
image_path = r'ortho/solar_pv_True_Ortho_Clip.tif'
coco_json_path = r'jsons\coco_json\runs\labelme2coco\dataset.json'
output_dir = "training_dataset_generated"

# Create folders
img_out_dir = os.path.join(output_dir, "images")
os.makedirs(img_out_dir, exist_ok=True)

# --- Load original image ---
image = cv2.imread(image_path)
if image is None:
    raise ValueError(f"Image not found at path: {image_path}")
H, W, _ = image.shape

# --- Load COCO annotations ---
with open(coco_json_path) as f:
    coco = json.load(f)

categories = coco["categories"]
anns = coco["annotations"]
img_info = coco["images"][0]
img_id = img_info["id"]

# --- Create new COCO structure ---
new_coco = {
    "images": [],
    "annotations": [],
    "categories": categories,
}
ann_id = 0
tile_id = 0

# --- Slide and create tiles ---
for y in tqdm(range(0, H, tile_size - overlap)):
    for x in range(0, W, tile_size - overlap):
        x1, y1 = x, y
        x2, y2 = min(x + tile_size, W), min(y + tile_size, H)
        tile = image[y1:y2, x1:x2]

        tile_annotations = []

        for ann in anns:
            if ann["image_id"] != img_id:
                continue

            seg = ann["segmentation"][0]
            poly = np.array(seg).reshape(-1, 2)

            if np.any((poly[:, 0] >= x1) & (poly[:, 0] <= x2) &
                      (poly[:, 1] >= y1) & (poly[:, 1] <= y2)):

                new_poly = (poly - [x1, y1]).reshape(-1).tolist()
                xmin, ymin = np.min(poly, axis=0) - [x1, y1]
                xmax, ymax = np.max(poly, axis=0) - [x1, y1]
                area = (xmax - xmin) * (ymax - ymin)

                tile_annotations.append({
                    "id": int(ann_id),
                    "image_id": int(tile_id),
                    "category_id": int(ann["category_id"]),
                    "segmentation": [list(map(float, new_poly))],
                    "bbox": [float(xmin), float(ymin), float(xmax - xmin), float(ymax - ymin)],
                    "iscrowd": 0,
                    "area": float(area)
                })
                ann_id += 1

        if tile_annotations:
            tile_filename = f"tile_{tile_id}.jpg"
            tile_path = os.path.join(img_out_dir, tile_filename)
            cv2.imwrite(tile_path, tile)

            new_coco["images"].append({
                "id": int(tile_id),
                "width": int(x2 - x1),
                "height": int(y2 - y1),
                "file_name": tile_filename
            })

            new_coco["annotations"].extend(tile_annotations)
            tile_id += 1

# --- Save new COCO JSON ---
with open(os.path.join(output_dir, "annotations.json"), "w") as f:
    json.dump(new_coco, f, indent=2)


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:01<00:00, 25.61it/s]


In [ ]:
#split data into train and validation sets. It will only split json files which detectron by default expects. Images are splitted in next code.

In [2]:
import json
import random
import os

# --- Paths ---
input_json_path = "training_dataset_generated/annotations.json"
output_dir = "training_dataset_generated/training_sets/annotations"
os.makedirs(output_dir, exist_ok=True)

# --- Load COCO JSON ---
with open(input_json_path, 'r') as f:
    coco = json.load(f)

# --- Shuffle and split images ---
random.seed(42)
images = coco["images"]
random.shuffle(images)

split_index = int(0.8 * len(images))
train_images = images[:split_index]
val_images = images[split_index:]

# --- Map image_id to images ---
train_ids = {img["id"] for img in train_images}
val_ids = {img["id"] for img in val_images}

# --- Split annotations ---
train_anns = [ann for ann in coco["annotations"] if ann["image_id"] in train_ids]
val_anns = [ann for ann in coco["annotations"] if ann["image_id"] in val_ids]

# --- Build final COCO JSONs ---
train_coco = {
    "images": train_images,
    "annotations": train_anns,
    "categories": coco["categories"]
}
val_coco = {
    "images": val_images,
    "annotations": val_anns,
    "categories": coco["categories"]
}

# --- Save output ---
with open(os.path.join(output_dir, "instances_train.json"), "w") as f:
    json.dump(train_coco, f, indent=2)

with open(os.path.join(output_dir, "instances_val.json"), "w") as f:
    json.dump(val_coco, f, indent=2)

print(f"Done! Train: {len(train_images)} images, Val: {len(val_images)} images")


✅ Done! Train: 111 images, Val: 28 images


In [ ]:
#split the images into train and valid folders.

In [5]:
import os
import shutil
import json

# Paths
img_dir = "training_dataset_generated/images"
ann_dir = "training_dataset_generated/training_sets/annotations"
train_dir = "training_dataset_generated/training_sets/train_images"
val_dir = "training_dataset_generated/training_sets/val_images"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Load train and val JSONs
with open(os.path.join(ann_dir, "instances_train.json")) as f:
    train_json = json.load(f)
with open(os.path.join(ann_dir, "instances_val.json")) as f:
    val_json = json.load(f)

# Move images
for img in train_json["images"]:
    src = os.path.join(img_dir, img["file_name"])
    dst = os.path.join(train_dir, img["file_name"])
    shutil.copyfile(src, dst)

for img in val_json["images"]:
    src = os.path.join(img_dir, img["file_name"])
    dst = os.path.join(val_dir, img["file_name"])
    shutil.copyfile(src, dst)

print("Images moved to train_images/ and val_images/")


Images moved to train_images/ and val_images/
